# Answering Business Questions Using SQL

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

### Overview of the Data
Let's query the database to get a list of all tables and views in our database. We will also query a few tables in the database.

In [2]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
select *
from customer
limit 5;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [5]:
%%sql
select *
from invoice
limit 5;

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


### Selecting Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and we need to select the first three albums that will be added to the store, from a list of four.

All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|----------------------|---------|
| Artist Name          | Genre   |
|----------------------|---------|
| Regal	               | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We'll write a query to find out which genres sell the most tracks in the USA.

In [10]:
%%sql
select g.name as genre,
       count(t.track_id) as num_tracks,
       round(count(t.track_id) * 100.0/(select count(*)
                          from track), 2) as percentage_total
from genre g
join track t
on g.genre_id = t.genre_id
join invoice_line il
on t.track_id = il.track_id
join invoice i
on i.invoice_id = il.invoice_id
where i.billing_country = 'USA'
group by genre
order by percentage_total desc;

 * sqlite:///chinook.db
Done.


genre,num_tracks,percentage_total
Rock,561,16.01
Alternative & Punk,130,3.71
Metal,124,3.54
R&B/Soul,53,1.51
Blues,36,1.03
Alternative,35,1.0
Pop,22,0.63
Latin,22,0.63
Hip Hop/Rap,20,0.57
Jazz,14,0.4


From our query, the top 5 genres in the US are Rock, Alternative & Punk, Metal, R&B/Soul and Blues. The Artist Red Tone (genre - Punk) should definitely be a priority purchase for the store.

Based on ranking, Chinook store should also purchase Slim Jim Bites (genre - Blues) and Meteor and the Girls (genre - Pop) for the store as the genres from the list where they received more sales activity.

### Analyzing Employee Sales Performance
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We will analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

We'll write a query that finds the total dollar amount of sales assigned to each sales support agent within the company, as well as any extra attributes for that employee that are relevant to the analysis.

In [16]:
%%sql
select e.first_name || ' ' || e.last_name as employee_name,
       e.title as employee_title,
       round(sum(i.total), 2) as total_sales,
       round(avg(i.total), 2) as avg_sales,
       count(distinct i.customer_id) as total_customers,
       count(i.invoice_id) as total_invoices,
       e2.first_name || ' ' || e2.last_name as manager_name,
       e2.title as supervisor_title
from invoice i
join customer c
on c.customer_id = i.customer_id
join employee e
on e.employee_id = c.support_rep_id
left join employee e2
on e.reports_to = e2.employee_id
group by c.support_rep_id, employee_name
order by total_sales desc;

 * sqlite:///chinook.db
Done.


employee_name,employee_title,total_sales,avg_sales,total_customers,total_invoices,manager_name,supervisor_title
Jane Peacock,Sales Support Agent,1731.51,8.17,21,212,Nancy Edwards,Sales Manager
Margaret Park,Sales Support Agent,1584.0,7.4,20,214,Nancy Edwards,Sales Manager
Steve Johnson,Sales Support Agent,1393.92,7.41,18,188,Nancy Edwards,Sales Manager



There are 3 sales support agents in our employee roster. All of them report to Sales Manager, Nancy Edwards. Jane Peacock is the employee who has made the most sales (1731.51USD from 21 customers), although Margaret Park has slightly more invoices than her (1584USD from 20 customers).

While having nearly the same amount of customers, Margaret Park's total sales are $148 less than Jane Peacock's. This means that Ms. Park's customer make smaller total purchases than Ms. Peacock's. This is confirmed by the average sales.

Steve Johnson makes significantly less in total_sales from a significantly fewer number of total invoices than the rest, though with only slightly fewer customers.

In [17]:
%%sql
select i.billing_country as country,
       round(sum(i.total), 2) as total_sales,
       count(i.customer_id) as num_customers,
       round(avg(i.total), 2) as avg_sales
       -- round(avg(il.quantity * il.unit_price), 2) as avg_order_value
from invoice i
join invoice_line il
on i.invoice_id = il.invoice_id
group by country
order by num_customers;

 * sqlite:///chinook.db
Done.


country,total_sales,num_customers,avg_sales
Denmark,196.02,38,5.16
Argentina,396.0,40,9.9
Italy,468.27,51,9.18
Belgium,567.27,61,9.3
Netherlands,544.5,66,8.25
Austria,649.44,70,9.28
Norway,664.29,73,9.1
Sweden,754.38,76,9.93
Poland,690.03,77,8.96
Hungary,830.61,79,10.51


### Analyzing Sales by Country
The next task is to analyze the sales data for customers from each different country. We will be using the country value from the customers table, and ignore the country from the billing address in the invoice table.

We'll write a query that collates data on purchases from different countries. Where a country has only one customer, collect them into an "Other" group.

The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
For each country, we'll include:
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [18]:
%%sql
drop view if exists  country_customer_table;

create view country_customer_table as
    select c.country as country,
           round(sum(i.total), 2) as total_sales,
           sum(il.unit_price) as total_value_sales,
           count(distinct c.customer_id) as num_customers,
           count(i.customer_id) as num_invoices,
           round(avg(i.total), 2) as avg_sales,
           round(sum(il.unit_price) / count(distinct i.invoice_id), 2) as average_order,
           round(sum(il.unit_price) / count(distinct i.customer_id), 2) as customer_lifetime_value
    from invoice i
    join invoice_line il
    on i.invoice_id = il.invoice_id
    join customer c
    on c.customer_id = i.customer_id
    group by country
    order by num_customers;

select
    case
        when cct.num_customers = 1 then 'Other'
        else cct.country
        end as country_group,
    round(sum(cct.total_sales), 2) as total_sales,
    round(sum(cct.total_value_sales), 2) as total_value_sales,
    sum(cct.num_customers) as num_customers,
    avg(cct.avg_sales) as  avg_sales,
    avg(cct.average_order) as avg_order,
    round(avg(cct.customer_lifetime_value), 2) as customer_lifetime_value
from country_customer_table cct
group by country_group
order by total_sales desc;

 * sqlite:///chinook.db
Done.
Done.
Done.


country_group,total_sales,total_value_sales,num_customers,avg_sales,avg_order,customer_lifetime_value
Other,10808.82,1094.94,15,9.476,7.444,73.0
USA,10405.89,1040.49,13,9.9,7.94,80.04
Canada,5489.55,535.59,8,10.15,7.05,66.95
Brazil,4059.0,427.68,5,9.4,7.01,85.54
France,3972.87,389.07,5,10.11,7.78,77.81
Germany,3441.24,334.62,4,10.18,8.16,83.66
Czech Republic,3183.84,273.24,2,11.54,9.11,136.62
United Kingdom,2498.76,245.52,3,10.08,8.77,81.84
India,1887.93,183.15,2,10.21,8.72,91.58
Portugal,1755.27,185.13,2,9.39,6.38,92.57


From the total_sales column, we see that USA, Canada and Brazil are the top 3 countries with the highest sales. In fact, the country_group of 'Other' which contains countries with only 1 customer, has as much sales as USA alone, and only a slightly higher number of customers.

USA sales and customers are nearly double the size of the second highest country, Canada. This means that USA is a significant single market.

### Album vs Individual Tracks
We will find out what percentage of purchases are individual tracks vs whole albums.

We will ignore the following cases:
* Albums that have only one or two tracks which are likely to be purchased by customers as part of a collection of individual tracks.
* Customers deciding to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In order to answer the question,we will identify whether each invoice has all the tracks from an album. We do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album.

We'll use EXCEPT to compare table values. We can check whether the exception of two sub queries IS NULL. Once we've made the comparison, we can wrap it in a CASE statement to add a column that tells us if that invoice was an album purchase or not.

We'll write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
* Number of invoices
* Percentage of invoices

### Next Steps
* Which artist is used in the most playlists?
* How many tracks have been purchased vs not purchased?
* Is the range of tracks in the store reflective of their sales popularity?
* Do protected vs non-protected media types have an effect on popularity?